<a href="https://colab.research.google.com/github/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/Chapter_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 13. 듀얼리티의 활용
## 13.1 들어가며

* 듀얼리티(duality)
  * 머신러닝을 통해 특정 도메인의 데이터 X를 받아 다른 도메인의 데이터 Y로 맵핑하는 함수를 근사
  * 대부분의 머신러닝 데이터셋은 두 도메인 사이의 데이터로 구성된다.(오디오, 텍스트)

  <img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/image1.png?raw=1" width="50%" height="50%">
